In [6]:
import pandas as pd

train_data = pd.read_csv('train2.csv', encoding = 'cp949')
test_data = pd.read_csv('test2.csv' , encoding = 'cp949')

In [3]:
# 0.66186

In [9]:
############ 1 
# 필요한 라이브러리 임포트
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
import lightgbm as lgb  # LightGBM 모델

# 1. 데이터 준비 및 전처리
# 'train_data' DataFrame에서 ID, 시술 시기 코드, 시술 유형 칼럼을 제외한 나머지가 feature, 
# '임신 성공 여부'가 target입니다.

# (1) 먼저 다운 샘플링을 위해 원본 DataFrame을 클래스별로 분리합니다.
df_majority = train_data[train_data['임신 성공 여부'] == 0]
df_minority = train_data[train_data['임신 성공 여부'] == 1]

# (2) 다수 클래스(0)를 소수 클래스(1)의 수와 맞춰 다운 샘플링
df_majority_downsampled = resample(
    df_majority,
    replace=False,                  # 중복 없이 샘플링
    n_samples=len(df_minority),     # 소수 클래스와 동일한 수
    random_state=42                 # 재현성을 위한 random_state 설정
)

# (3) 다운 샘플링된 데이터와 소수 클래스 데이터를 결합
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

# 2. 피처와 타겟 변수 분리
# 제외할 칼럼: 'ID', '시술 시기 코드', '시술 유형', 그리고 '임신 성공 여부'는 target이므로 제외
cols_to_drop = ['ID', '시술 시기 코드', '시술 유형', '임신 성공 여부', 'ID', '시술 시기 코드', '시술 유형', '임신 성공 여부','여성 주 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', 'IVF 시술 횟수', 'IVF 출산 횟수', '혼합된 난자 수', '동결 배아 사용 여부', '신선 배아 사용 여부']
X = df_downsampled.drop(columns=cols_to_drop)
y = df_downsampled['임신 성공 여부']

# 3. 학습/검증 데이터 분리 (Stratify 옵션으로 클래스 비율 유지)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 4. 모델 학습 (LightGBM 사용)
clf = lgb.LGBMClassifier(random_state=42)
clf.fit(X_train, y_train)

# 5. 예측 및 평가
y_pred = clf.predict(X_val)
y_prob = clf.predict_proba(X_val)[:, 1]  # Positive class 확률

# 평가 지표 출력
accuracy = accuracy_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_prob)
conf_mat = confusion_matrix(y_val, y_pred)

print("Accuracy: {:.4f}".format(accuracy))
print("ROC-AUC: {:.4f}".format(roc_auc))
print("Confusion Matrix:\n", conf_mat)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 45783, number of negative: 45782
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.031551 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 160
[LightGBM] [Info] Number of data points in the train set: 91565, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500005 -> initscore=0.000022
[LightGBM] [Info] Start training from score 0.000022
Accuracy: 0.6636
ROC-AUC: 0.7185
Confusion Matrix:
 [[10498  9124]
 [ 4078 15543]]


In [25]:
# 6. 다른 threshold 값에 따른 평가 지표 출력
thresholds = [0.55, 0.6, 0.65]

for thresh in thresholds:
    # 해당 threshold 이상이면 1로 분류
    y_pred_thresh = (y_prob >= thresh).astype(int)
    
    # 평가 지표 계산
    acc = accuracy_score(y_val, y_pred_thresh)
    roc_auc = roc_auc_score(y_val, y_prob)  # ROC-AUC는 확률값을 사용하므로 threshold와 무관
    conf_mat = confusion_matrix(y_val, y_pred_thresh)
    
    print(f"Threshold: {thresh}")
    print("Accuracy: {:.4f}".format(acc))
    print("ROC-AUC: {:.4f}".format(roc_auc))
    print("Confusion Matrix:\n", conf_mat)
    print("F1 score : {:.4f}".format(f1_score(y_val,y_pred)))
    print("-" * 50)

Threshold: 0.55
Accuracy: 0.6550
ROC-AUC: 0.7185
Confusion Matrix:
 [[12334  7288]
 [ 6251 13370]]
F1 score : 0.7019
--------------------------------------------------
Threshold: 0.6
Accuracy: 0.6416
ROC-AUC: 0.7185
Confusion Matrix:
 [[13848  5774]
 [ 8292 11329]]
F1 score : 0.7019
--------------------------------------------------
Threshold: 0.65
Accuracy: 0.6081
ROC-AUC: 0.7185
Confusion Matrix:
 [[15870  3752]
 [11626  7995]]
F1 score : 0.7019
--------------------------------------------------


In [21]:
from sklearn.metrics import recall_score, f1_score
recall = recall_score(y_val, y_pred)
recall

0.7921614596605677

In [15]:
# 테스트 데이터 불러오기 (cp949 인코딩)
test_data = pd.read_csv('test2.csv', encoding = 'cp949')

# 테스트 데이터 전처리
# 학습 시 사용했던 제외할 칼럼과 동일하게 제거합니다.
cols_to_drop = ['ID', '시술 시기 코드', '시술 유형', '임신 성공 여부', 'ID', '시술 시기 코드', '시술 유형', '임신 성공 여부','여성 주 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', 'IVF 시술 횟수', 'IVF 출산 횟수', '혼합된 난자 수', '동결 배아 사용 여부', '신선 배아 사용 여부']
X_test = test_data.drop(columns=cols_to_drop)

# 예측 진행
test_pred = clf.predict(X_test)
test_prob = clf.predict_proba(X_test)[:, 1]  # 양성 클래스(1)에 대한 확률

# 예측 결과를 테스트 데이터에 추가 (원하는 경우)
test_data['임신 성공 여부 예측'] = test_pred
test_data['임신 성공 여부 예측_확률'] = test_prob

# 결과 확인
print(test_data[['ID', '임신 성공 여부 예측', '임신 성공 여부 예측_확률']].head())

# 예측 결과를 CSV 파일로 저장 (원하는 경우)
test_data.to_csv('lgbm_test_ivf_columns_selection_predictions_0224.csv', index=False, encoding='cp949')

           ID  임신 성공 여부 예측  임신 성공 여부 예측_확률
0  TEST_00000            0        0.001347
1  TEST_00001            0        0.000835
2  TEST_00002            0        0.369695
3  TEST_00003            0        0.270678
4  TEST_00004            1        0.645813


In [54]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
import lightgbm as lgb  # LightGBM 모델

# 1. 데이터 준비 및 전처리
# 'train_data' DataFrame에서 ID, 시술 시기 코드, 시술 유형 칼럼을 제외한 나머지가 feature, 
# '임신 성공 여부'가 target입니다.

# (1) 먼저 다운 샘플링을 위해 원본 DataFrame을 클래스별로 분리합니다.
df_majority = train_data[train_data['임신 성공 여부'] == 0]
df_minority = train_data[train_data['임신 성공 여부'] == 1]

# (2) 다수 클래스(0)를 소수 클래스(1)의 수와 맞춰 다운 샘플링
df_majority_downsampled = resample(
    df_majority,
    replace=False,                  # 중복 없이 샘플링
    n_samples=len(df_minority),     # 소수 클래스와 동일한 수
    random_state=42                 # 재현성을 위한 random_state 설정
)

# (3) 다운 샘플링된 데이터와 소수 클래스 데이터를 결합
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

# 2. 피처와 타겟 변수 분리
# 제외할 칼럼: 'ID', '시술 시기 코드', '시술 유형', '임신 성공 여부'(target) 칼럼 제거
cols_to_drop = ['ID', '시술 시기 코드', '시술 유형', '임신 성공 여부']
X = df_downsampled.drop(columns=cols_to_drop)
y = df_downsampled['임신 성공 여부']

# 3. 학습/검증 데이터 분리 (Stratify 옵션으로 클래스 비율 유지)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 4. 하이퍼파라미터 튜닝: GridSearchCV를 사용하여 최적의 파라미터 탐색
param_grid = {
    'n_estimators': [300],
    'learning_rate': [0.001, 0.01, 0.05],
    'num_leaves': [50, 70],
    'max_depth': [30]
}

lgb_estimator = lgb.LGBMClassifier(random_state=42)

grid_search = GridSearchCV(
    estimator=lgb_estimator,
    param_grid=param_grid,
    scoring='accuracy',  # 또는 roc_auc 등 다른 평가지표 사용 가능
    cv=3,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

print("Best Parameters: ", grid_search.best_params_)

# 5. 최적 모델로 검증 데이터 예측 및 평가
best_clf = grid_search.best_estimator_

y_pred = best_clf.predict(X_val)
y_prob = best_clf.predict_proba(X_val)[:, 1]  # 양성 클래스(1)의 확률

# 평가 지표 계산
accuracy = accuracy_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_prob)
conf_mat = confusion_matrix(y_val, y_pred)

print("Accuracy: {:.4f}".format(accuracy))
print("ROC-AUC: {:.4f}".format(roc_auc))
print("Confusion Matrix:\n", conf_mat)

Fitting 3 folds for each of 81 candidates, totalling 243 fits
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 45783, number of negative: 45782
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.056121 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 230
[LightGBM] [Info] Number of data points in the train set: 91565, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500005 -> initscore=0.000022
[LightGBM] [Info] Start training from score 0.000022
Best Parameters:  {'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 300, 'num_leaves': 50}
Accuracy: 0.6629
ROC-AUC: 0.7189
Confusion Matrix:
 [[10508  9114]
 [ 4115 15506]]


In [8]:
###############2



import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
import lightgbm as lgb  # LightGBM 모델

# 1. 데이터 준비 및 전처리
# 'train_data' DataFrame에서 ID, 시술 시기 코드, 시술 유형 칼럼을 제외한 나머지가 feature, 
# '임신 성공 여부'가 target입니다.

# (1) 먼저 다운 샘플링을 위해 원본 DataFrame을 클래스별로 분리합니다.
df_majority = train_data[train_data['임신 성공 여부'] == 0]
df_minority = train_data[train_data['임신 성공 여부'] == 1]

# (2) 다수 클래스(0)를 소수 클래스(1)의 수와 맞춰 다운 샘플링
df_majority_downsampled = resample(
    df_majority,
    replace=False,                  # 중복 없이 샘플링
    n_samples=len(df_minority),     # 소수 클래스와 동일한 수
    random_state=42                 # 재현성을 위한 random_state 설정
)

# (3) 다운 샘플링된 데이터와 소수 클래스 데이터를 결합
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

# 2. 피처와 타겟 변수 분리
# 제외할 칼럼: 'ID', '시술 시기 코드', '시술 유형', '임신 성공 여부'(target) 칼럼 제거
cols_to_drop = ['ID', '시술 시기 코드', '시술 유형', '임신 성공 여부', 'ID', '시술 시기 코드', '시술 유형', '임신 성공 여부','여성 주 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', 'IVF 시술 횟수', 'IVF 출산 횟수', '혼합된 난자 수', '동결 배아 사용 여부', '신선 배아 사용 여부']
X = df_downsampled.drop(columns=cols_to_drop)
y = df_downsampled['임신 성공 여부']

# 3. 학습/검증 데이터 분리 (Stratify 옵션으로 클래스 비율 유지)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 4. 하이퍼파라미터 튜닝: GridSearchCV를 사용하여 최적의 파라미터 탐색
param_grid = {
    'n_estimators': [500],
    'learning_rate': [0.01],
    'num_leaves': [70],
    'max_depth': [20]
}

lgb_estimator = lgb.LGBMClassifier(random_state=42)

grid_search = GridSearchCV(
    estimator=lgb_estimator,
    param_grid=param_grid,
    scoring='accuracy',  # 또는 roc_auc 등 다른 평가지표 사용 가능
    cv=3,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

print("Best Parameters: ", grid_search.best_params_)

# 5. 최적 모델로 검증 데이터 예측 및 평가
best_clf = grid_search.best_estimator_

y_pred = best_clf.predict(X_val)
y_prob = best_clf.predict_proba(X_val)[:, 1]  # 양성 클래스(1)의 확률

# 평가 지표 계산
accuracy = accuracy_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_prob)
conf_mat = confusion_matrix(y_val, y_pred)

print("Accuracy: {:.4f}".format(accuracy))
print("ROC-AUC: {:.4f}".format(roc_auc))
print("Confusion Matrix:\n", conf_mat)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 45783, number of negative: 45782
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017683 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 160
[LightGBM] [Info] Number of data points in the train set: 91565, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500005 -> initscore=0.000022
[LightGBM] [Info] Start training from score 0.000022
Best Parameters:  {'learning_rate': 0.01, 'max_depth': 20, 'n_estimators': 500, 'num_leaves': 70}
Accuracy: 0.6632
ROC-AUC: 0.7182
Confusion Matrix:
 [[10500  9122]
 [ 4095 15526]]


In [56]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
import lightgbm as lgb  # LightGBM 모델

# 1. 데이터 준비 및 전처리
# 'train_data' DataFrame에서 ID, 시술 시기 코드, 시술 유형 칼럼을 제외한 나머지가 feature, 
# '임신 성공 여부'가 target입니다.

# (1) 먼저 다운 샘플링을 위해 원본 DataFrame을 클래스별로 분리합니다.
df_majority = train_data[train_data['임신 성공 여부'] == 0]
df_minority = train_data[train_data['임신 성공 여부'] == 1]

# (2) 다수 클래스(0)를 소수 클래스(1)의 수와 맞춰 다운 샘플링
df_majority_downsampled = resample(
    df_majority,
    replace=False,                  # 중복 없이 샘플링
    n_samples=len(df_minority),     # 소수 클래스와 동일한 수
    random_state=42                 # 재현성을 위한 random_state 설정
)

# (3) 다운 샘플링된 데이터와 소수 클래스 데이터를 결합
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

# 2. 피처와 타겟 변수 분리
# 제외할 칼럼: 'ID', '시술 시기 코드', '시술 유형', '임신 성공 여부'(target) 칼럼 제거
cols_to_drop = ['ID', '시술 시기 코드', '시술 유형', '임신 성공 여부']
X = df_downsampled.drop(columns=cols_to_drop)
y = df_downsampled['임신 성공 여부']

# 3. 학습/검증 데이터 분리 (Stratify 옵션으로 클래스 비율 유지)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 4. 하이퍼파라미터 튜닝: GridSearchCV를 사용하여 최적의 파라미터 탐색
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [31, 50, 70],
    'max_depth': [-1, 10, 20]
}

lgb_estimator = lgb.LGBMClassifier(random_state=42)

grid_search = GridSearchCV(
    estimator=lgb_estimator,
    param_grid=param_grid,
    scoring='roc_auc',  # 또는 roc_auc 등 다른 평가지표 사용 가능
    cv=3,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

print("Best Parameters: ", grid_search.best_params_)

# 5. 최적 모델로 검증 데이터 예측 및 평가
best_clf = grid_search.best_estimator_

y_pred = best_clf.predict(X_val)
y_prob = best_clf.predict_proba(X_val)[:, 1]  # 양성 클래스(1)의 확률

# 평가 지표 계산
accuracy = accuracy_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_prob)
conf_mat = confusion_matrix(y_val, y_pred)

print("Accuracy: {:.4f}".format(accuracy))
print("ROC-AUC: {:.4f}".format(roc_auc))
print("Confusion Matrix:\n", conf_mat)

Fitting 3 folds for each of 81 candidates, totalling 243 fits
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 45783, number of negative: 45782
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007310 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 230
[LightGBM] [Info] Number of data points in the train set: 91565, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500005 -> initscore=0.000022
[LightGBM] [Info] Start training from score 0.000022
Best Parameters:  {'learning_rate': 0.01, 'max_depth': -1, 'n_estimators': 300, 'num_leaves': 31}
Accuracy: 0.6640
ROC-AUC: 0.7191
Confusion Matrix:
 [[10484  9138]
 [ 4047 15574]]


In [58]:
# 테스트 데이터 불러오기 (cp949 인코딩)
test_data = pd.read_csv('test2.csv', encoding = 'cp949')

# 테스트 데이터 전처리
# 학습 시 사용했던 제외할 칼럼과 동일하게 제거합니다.
cols_to_drop = ['ID', '시술 시기 코드', '시술 유형', '임신 성공 여부']
X_test = test_data.drop(columns=cols_to_drop)

# 예측 진행
test_pred = clf.predict(X_test)
test_prob = clf.predict_proba(X_test)[:, 1]  # 양성 클래스(1)에 대한 확률

# 예측 결과를 테스트 데이터에 추가 (원하는 경우)
test_data['임신 성공 여부 예측'] = test_pred
test_data['임신 성공 여부 예측_확률'] = test_prob

# 결과 확인
print(test_data[['ID', '임신 성공 여부 예측', '임신 성공 여부 예측_확률']].head())

# 예측 결과를 CSV 파일로 저장 (원하는 경우)
test_data.to_csv('test_ivf_lgbm_tuned_predictions.csv', index=False, encoding='cp949')

           ID  임신 성공 여부 예측  임신 성공 여부 예측_확률
0  TEST_00000            0        0.001504
1  TEST_00001            0        0.000689
2  TEST_00002            0        0.383503
3  TEST_00003            0        0.257484
4  TEST_00004            1        0.648965
